In [3]:
# Toggles
USE_FEATURE_ENGINEERING = False
USE_TRANSFORMATION = True
USE_SCALER = True
USE_WEIGHTED_RMSE = True
USE_RFR_FEATURE_SELECTION = False

# Protected Columns
exclude_columns = ['reiwa_price', 'identifier', 'reiwa_is_sold']



In [4]:
# Imports
import numpy as np
import pandas as pd
import xgboost as xgb
import json
import itertools
from tqdm.notebook import tqdm
from scipy.stats import boxcox
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from scipy.special import inv_boxcox
from collections.abc import MutableMapping
from IPython.core.magic import (register_line_magic, magics_class, Magics)
from IPython.display import display, Javascript
from plyer import notification


In [5]:
# Function to send a notification
def notify(title, message):
    notification.notify(
        title=title,
        message=message,
        app_name='Jupyter Notebook'
    )

# Display a notification when the cell completes execution
def notify_when_done(title='Task Complete', message='The cell execution is complete.'):
    display(Javascript('''
        (async () => {
            await new Promise(resolve => {
                setTimeout(resolve, 100);
            });
            await google.colab.kernel.invokeFunction('notebook.notify_completion', [], {});
        })();
    '''))
    get_ipython().register_magics(CompletionNotifier)

@magics_class
class CompletionNotifier(Magics):
    @register_line_magic
    def notify_completion(self, line):
        notify(title, message)

get_ipython().register_magics(CompletionNotifier)

In [6]:
# Function to flatten nested dictionaries
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Load and process property data
with open('property_data.json', 'r') as file:
    property_data = json.load(file)
property_df = pd.DataFrame(property_data)
property_df.fillna(0, inplace=True)
property_df['identifier'] = range(1, len(property_df) + 1)

# Load and process suburb data
with open('suburb_data.json', 'r') as file:
    suburb_data = json.load(file)
required_suburbs = set(property_df['reiwa_suburb'])
filtered_suburb_data = {k: flatten_dict(v) for k, v in suburb_data.items() if k in required_suburbs}
suburb_df = pd.DataFrame.from_dict(filtered_suburb_data, orient='index').reset_index().rename(columns={'index': 'reiwa_suburb'})
suburb_df.fillna(0, inplace=True)

# Filter property dataframe
property_df = (
    property_df.drop_duplicates(subset='reiwa_listing_id', keep='first')
    .assign(reiwa_price=lambda x: x[['reiwa_listing_price', 'reiwa_price']].max(axis=1))
    .drop(columns=['reiwa_listing_price'])
    .query('osm_local_community_population != 0')
)

# Create total rooms count for various filtering tasks
property_df['total_rooms'] = property_df['reiwa_bedrooms'] + property_df['reiwa_bathrooms'] + property_df['reiwa_parking']

# Apply scaling logic
property_df.loc[(property_df['reiwa_house_type'] == 'Land') & (property_df['reiwa_landsize'] < 100), 'reiwa_landsize'] *= 10000
property_df.loc[(property_df['reiwa_house_type'] != 'Land') & (property_df['reiwa_landsize'] < 40) & (property_df['total_rooms'] > 4), 'reiwa_landsize'] *= 10000

# Sanity check to filter out unrealistic properties
min_price_per_sqm = 1000
max_price_per_sqm = 100000
conditions = (
    (property_df['reiwa_price'] <= 10000000) &  # Property price should be less than or equal to 10 million
    (property_df['reiwa_price'] >= 50000) &  # Property price should be greater than or equal to 50k
    (property_df['reiwa_landsize'] > 25) & (property_df['reiwa_landsize'] <= 10000) &  # Land size should be between 25 sqm and 10,000 sqm
    ((property_df['total_rooms'] == 0) | (property_df['reiwa_landsize'] / property_df['total_rooms'] >= 10)) &  # If no rooms, condition is met; else, land size per room should be at least 10 sqm
    ((property_df['reiwa_bedrooms'] == 0) | (property_df['reiwa_price'] / property_df['reiwa_bedrooms'] >= 50000)) &  # If no bedrooms, condition is met; else, price per bedroom should be at least 50k
    ((property_df['total_rooms'] > 0) |  # If total rooms are more than 0,
    ((property_df['reiwa_price'] / property_df['reiwa_landsize'] >= min_price_per_sqm) &  # price per sqm should be at least 1k
    (property_df['reiwa_price'] / property_df['reiwa_landsize'] <= max_price_per_sqm)))  # and not exceed 100k
)
property_df = property_df[conditions].drop(columns=['total_rooms'])

# Filter suburb dataframe
suburb_df = suburb_df.query('reiwa_suburb_interest_level.notnull()')

# Merge property and suburb data
df = pd.merge(property_df, suburb_df, on='reiwa_suburb', how='left')

# Make 'identifier' the first column
cols = ['identifier'] + [col for col in df.columns if col != 'identifier']
df = df[cols]

# Fill NaNs with zero (we have many optional columns)
df = df.apply(lambda col: col.fillna(col.median()) if col.dtype != 'O' else col)

# Drop rows where 'suburb_interest_level' is null (indicates no valid REIWA data)
df = df[df['reiwa_suburb_interest_level'].notnull()]

# Drop text-based fields that are not being encoded
text_fields = ['reiwa_address', 'reiwa_image_url', 'reiwa_details_url', 'reiwa_agency_name']
df = df.drop(columns=text_fields)

# Drop text-based fields that are not being encoded
list_fields = ['abs_coordinates']
df = df.drop(columns=list_fields)

# One-hot encode categorical fields
df = pd.get_dummies(df, columns=['reiwa_agency_no', 'reiwa_suburb', 'reiwa_house_type', 'reiwa_local_government', 'scsa_school'])



In [7]:
if USE_FEATURE_ENGINEERING:
    def shannon_diversity_index(proportions):
        proportions = proportions[proportions > 0]  # Filter out zero values
        return -np.sum(proportions * np.log(proportions))

    # Property and Land Features
    df['price_per_square_meter'] = df['reiwa_price'] / df['reiwa_landsize']
    df['bedrooms_to_bathrooms_ratio'] = df['reiwa_bedrooms'] / df['reiwa_bathrooms']
    df['total_parking_spaces'] = df['reiwa_parking'] + df['reiwa_bedrooms']
    df['land_value_ratio'] = df['reiwa_price'] / df['reiwa_median_house_sale']
    df['distance_to_cbd_airport_ratio'] = df['osm_distance_to_perth_cbd'] / df['osm_distance_to_perth_airport']
    df['price_vs_prediction'] = df['reiwa_price'] - df['model_prediction']
    df['rooms_per_square_meter'] = (df['reiwa_bedrooms'] + df['reiwa_bathrooms']) / df['reiwa_landsize']

    # Community and Amenities (within 1.5 km radius)
    radius_area_km2 = 3.14 * (1.5 ** 2)  # Area of a circle with radius 1.5 km

    df['population_density_within_1_5km'] = df['osm_local_community_population'] / radius_area_km2
    df['dwellings_density_within_1_5km'] = df['osm_local_community_dwellings'] / radius_area_km2
    df['healthcare_accessibility_index'] = df['osm_nearest_healthcare_facility'] / df['abs_people']
    df['education_accessibility_index'] = df['osm_nearest_primary_education'] + df['osm_nearest_higher_education']
    df['public_service_density'] = (df['osm_nearest_fire_station'] + df['osm_nearest_police_station'] + df['osm_nearest_library']) / radius_area_km2
    df['recreation_density_within_1_5km'] = (df['osm_local_leisure_facility'] + df['osm_local_sports_facility']) / radius_area_km2
    df['proximity_to_public_transport'] = (df['osm_nearest_bus_stop'] + df['osm_nearest_train_station']) / 2

    # Crime and Safety
    df['person_crime_to_property_crime_ratio'] = df['wapol_total_person_crime'] / df['wapol_total_property_crime']
    df['crime_rate'] = (df['wapol_total_person_crime'] + df['wapol_total_property_crime']) / df['abs_people']
    df['violent_crime_rate'] = (df['wapol_offences_homicide'] + df['wapol_offences_sexual'] + df['wapol_offences_assault_family'] + df['wapol_offences_assault_non_family']) / df['abs_people']
    df['property_crime_rate'] = (df['wapol_offences_dwelling_burglary'] + df['wapol_offences_non_dwelling_burglary'] + df['wapol_offences_stealing'] + df['wapol_offences_property_damage'] + df['wapol_offences_arson']) / df['abs_people']
    df['crime_density'] = (df['wapol_total_person_crime'] + df['wapol_total_property_crime']) / df['abs_area_km2']

    # School and Education
    df['school_score_index'] = df['scsa_awper'] + df['scsa_cert2fouratarper'] + df['scsa_numendper'] + df['scsa_readendper'] + df['scsa_writeendpe']
    df['school_to_population_ratio'] = df['scsa_ewnum'] / df['osm_local_community_population']
    df['school_success_rate'] = df['scsa_awnum'] / df['scsa_ewnum']
    df['ATAR_success_rate'] = df['scsa_fouratarper'] / 100

    # Socioeconomic Indicators
    df['income_to_rent_ratio'] = df['abs_median_weekly_household_income'] / df['abs_median_weekly_rent']
    df['income_to_mortgage_ratio'] = df['abs_median_weekly_household_income'] / (df['abs_median_monthly_mortgage_repayment'] / 4)
    df['vehicle_ownership_rate'] = df['abs_avg_vehicles_per_house']
    df['unemployment_rate'] = df['abs_sub_employment_status_unemployed_pct']
    df['fulltime_employment_rate'] = df['abs_sub_employment_status_worked_fulltime_pct']

    # Demographic and Health
    df['elderly_population_ratio'] = (df['abs_sub_age_6569_years_val'] + df['abs_sub_age_7074_years_val'] + df['abs_sub_age_7579_years_val'] + df['abs_sub_age_8084_years_val'] + df['abs_sub_age_85_years_and_over_val']) / df['abs_people']
    df['youth_population_ratio'] = (df['abs_sub_age_04_years_val'] + df['abs_sub_age_59_years_val'] + df['abs_sub_age_1014_years_val'] + df['abs_sub_age_1519_years_val']) / df['abs_people']
    df['longterm_health_condition_rate'] = (df['abs_sub_type_of_longterm_health_condition_arthritis_val'] + df['abs_sub_type_of_longterm_health_condition_asthma_val'] + df['abs_sub_type_of_longterm_health_condition_cancer_val'] + df['abs_sub_type_of_longterm_health_condition_diabetes_val'] + df['abs_sub_type_of_longterm_health_condition_mental_health_condition_val']) / df['abs_people']
    df['no_health_condition_rate'] = df['abs_sub_type_of_longterm_health_condition_no_longterm_health_condition_pct']

    # Transportation and Commuting
    df['car_dependency_rate'] = df['abs_sub_method_of_travel_to_work_on_the_day_of_the_census_top_responses_people_who_travelled_to_work_by_car_as_driver_or_passenger_pct']
    df['public_transport_usage_rate'] = df['abs_sub_method_of_travel_to_work_on_the_day_of_the_census_top_responses_people_who_travelled_to_work_by_public_transport_pct']
    df['work_from_home_rate'] = df['abs_sub_method_of_travel_to_work_on_the_day_of_the_census_top_responses_worked_at_home_pct']
    df['non_commuting_population_rate'] = df['abs_sub_method_of_travel_to_work_on_the_day_of_the_census_top_responses_did_not_go_to_work_pct']

    # Family and Housing
    df['household_size'] = df['abs_people_per_household']
    df['children_per_family'] = df['abs_child_per_family']
    df['family_households_rate'] = df['abs_sub_household_composition_family_households_pct']
    df['single_person_households_rate'] = df['abs_sub_household_composition_single_person_households_pct']
    df['couple_with_children_rate'] = df['abs_sub_family_composition_couple_family_with_children_pct']
    df['owned_outright_rate'] = df['abs_sub_tenure_type_owned_outright_pct']
    df['owned_with_mortgage_rate'] = df['abs_sub_tenure_type_owned_with_a_mortgage_pct']
    df['rental_affordability_index'] = df['abs_sub_rent_weekly_payments_median_rent_val'] / df['abs_median_weekly_household_income']

    # New Features from Existing Data
    df['nearby_schools_quality_index'] = df['scsa_awper'] * df['scsa_readendper'] / 100
    df['crime_trend'] = df['wapol_avg_person_crime_prev_3y'] / df['wapol_total_person_crime']

    # Odd features
    df['price_anomaly_score'] = (df['reiwa_price'] - df['model_prediction']) / df['model_prediction']
    df['lat_long_interaction'] = df['reiwa_latitude'] * df['reiwa_longitude']
    df['bedrooms_to_land_size_ratio'] = df['reiwa_bedrooms'] / df['reiwa_landsize']
    df['bathrooms_to_land_size_ratio'] = df['reiwa_bathrooms'] / df['reiwa_landsize']
    df['parking_to_land_size_ratio'] = df['reiwa_parking'] / df['reiwa_landsize']
    df['coast_and_transport_score'] = df['osm_nearest_coast'] * df['proximity_to_public_transport']
    df['combined_education_accessibility'] = df['osm_nearest_primary_education'] + df['osm_nearest_higher_education'] + df['osm_nearest_library']
    df['combined_crime_rate'] = (df['wapol_total_person_crime'] + df['wapol_total_property_crime']) / df['abs_people']
    df['high_school_performance_index'] = df['ATAR_success_rate'] * df['school_score_index']
    df['affordability_index'] = df['income_to_rent_ratio'] + df['income_to_mortgage_ratio']
    df['employment_health_index'] = df['fulltime_employment_rate'] * df['no_health_condition_rate']
    df['suburb_density_ratio'] = df['abs_people'] / df['abs_houses']
    df['vehicle_to_person_ratio'] = df['abs_avg_vehicles_per_house'] / df['abs_people_per_household']
    df['gender_ratio'] = df['abs_male_ratio'] / df['abs_female_ratio']
    df['nearby_amenities_score'] = df[['osm_local_dining', 'osm_local_shop', 'osm_local_sports_facility',
                                    'osm_local_leisure_facility', 'osm_local_public_art']].sum(axis=1)
    df['crime_to_amenities_ratio'] = df['combined_crime_rate'] / df['nearby_amenities_score']
    df['suburb_socioeconomic_status'] = df['abs_median_weekly_household_income'] / df['abs_median_age']
    df['proximity_to_essential_services'] = df[['osm_nearest_healthcare_facility', 'osm_nearest_police_station',
                                                'osm_nearest_fire_station', 'osm_nearest_post_office',
                                                'osm_nearest_financial_services']].sum(axis=1)
    
    # Diversity Features
    religious_columns = [col for col in df.columns if col.startswith('abs_sub_religious_affiliation_top_responses') and col.endswith('_pct')]
    df['religious_diversity'] = df[religious_columns].apply(lambda row: shannon_diversity_index(row / 100), axis=1)

    # Find all columns related to cultural diversity
    cultural_columns = [col for col in df.columns if col.startswith('abs_sub_ancestry_top_responses') and col.endswith('_pct')]
    df['cultural_diversity'] = df[cultural_columns].apply(lambda row: shannon_diversity_index(row / 100), axis=1)

    # Display the resulting dataframe with new diversity features
    print(df[['religious_diversity', 'cultural_diversity']].head())



In [8]:
def drop_low_variance_bools(df, threshold=10):
    """
    Drop integer columns that only contain values of 0 and 1 and have very few 1 records.

    Parameters:
    df (pd.DataFrame): The input dataframe.
    threshold (int): The maximum number of '1' values allowed for the column to be dropped. Default is 10.

    Returns:
    pd.DataFrame: The dataframe with the specified columns dropped.
    """
    cols_to_drop = []
    for col in df.select_dtypes(include='bool').columns:
        if set(df[col].unique()).issubset({0, 1}):
            if df[col].sum() < threshold:
                cols_to_drop.append(col)
    
    df_dropped = df.drop(columns=cols_to_drop)
    return df_dropped

#df = drop_low_variance_bools(df, threshold=10)

def drop_columns_by_prefix(df, prefixes):
    """
    Drop columns that begin with any of the specified prefixes.

    Parameters:
    df (pd.DataFrame): The input dataframe.
    prefixes (list of str): The list of prefixes to check.

    Returns:
    pd.DataFrame: The dataframe with the specified columns dropped.
    """
    cols_to_drop = [col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]
    df_dropped = df.drop(columns=cols_to_drop)
    return df_dropped

prefixes = ["wapol_offences_"]
df = drop_columns_by_prefix(df, prefixes)

# Boxcox our target variable (must take place before scaler)
df['reiwa_price'], boxcox_lambda = boxcox(df['reiwa_price'])

# Function to determine if a column should use square root transformation
def should_use_sqrt(col):
    unique_values = df[col].nunique()
    return unique_values <= 20

# Collect transformable columns
transformable_columns = []

for col in df.columns:
    if col in exclude_columns or df[col].dtype == bool or set(df[col].unique()) == {0, 1}:
        continue
    
    # Debug: Print the minimum value of the column
    min_value = df[col].min()

    transformable_columns.append(col)

if USE_TRANSFORMATION:
    for col in transformable_columns:
        if (df[col] < 0).any():
            df[col] = df[col] * -1

        if should_use_sqrt(col):
            df[col] = np.sqrt(df[col])
        else:
            try:
                if (df[col] <= 0).any():
                    df[col] += abs(df[col].min()) + 1  # Shift to make all values positive
                df[col], _ = boxcox(df[col])
            except ValueError as e:
                continue

# Apply scaling if the toggle is on
if USE_SCALER:
    scaler = StandardScaler()
    df[transformable_columns] = scaler.fit_transform(df[transformable_columns])

In [9]:
def feature_selection_with_rfr(df, target, protected_columns, variance_threshold=0.01, feature_importance_threshold=0.01, n_estimators=50, max_depth=5):
    """
    Perform feature selection using Variance Threshold and Random Forest Regressor, retaining specified protected columns.
    
    Parameters:
    df (pd.DataFrame): The dataframe containing the data.
    target (str): The target column name.
    protected_columns (list): A list of columns that should not be dropped.
    variance_threshold (float): The threshold for variance to remove low-variance features.
    feature_importance_threshold (float): The threshold for feature importance. Features with importance below this value will be dropped.
    n_estimators (int): Number of trees in the forest.
    max_depth (int): Maximum depth of the tree.
    
    Returns:
    pd.DataFrame: The dataframe with unnecessary columns dropped.
    """
    # Separate features and target
    X = df.drop(columns=[target])
    y = df[target]
    
    # Apply Variance Threshold to reduce dimensionality
    vt = VarianceThreshold(threshold=variance_threshold)
    X_reduced = vt.fit_transform(X)
    
    # Get the reduced feature names
    reduced_features = X.columns[vt.get_support()]
    
    # Create a new dataframe with the reduced features
    X_reduced_df = pd.DataFrame(X_reduced, columns=reduced_features, index=X.index)
    
    # Initialize and fit Random Forest Regressor
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1, verbose=1)
    model.fit(X_reduced_df, y)
    
    # Select features based on importance
    selector = SelectFromModel(model, threshold=feature_importance_threshold, prefit=True)
    selected_features = X_reduced_df.columns[(selector.get_support())]
    
    # Ensure protected columns are retained
    columns_to_keep = set(selected_features).union(set(protected_columns))
    
    # Ensure all columns to keep are in the dataframe
    columns_to_keep = [col for col in columns_to_keep if col in df.columns]
    
    # Drop unnecessary columns directly from the original dataframe
    columns_to_drop = set(df.columns) - set(columns_to_keep) - {target}
    df.drop(columns=columns_to_drop, inplace=True)
    
    return df

if USE_RFR_FEATURE_SELECTION:
    df = feature_selection_with_rfr(df, 'reiwa_price', exclude_columns, max_depth=None, n_estimators=200, feature_importance_threshold=0.005)


In [10]:
# We only want to train on is_sold == True since we are predicting pre-sales
df_sold = df[df['reiwa_is_sold'] == True]

# We modify this dataframe later so we copy it to avoid warnings
df_unsold = df[df['reiwa_is_sold'] == False].copy()

# Split the sold data into features (X) and target variable (y), excluding 'identifier'
X = df_sold.drop(['reiwa_price', 'reiwa_is_sold', 'identifier'], axis=1)
y = df_sold['reiwa_price']

# Perform the train-test split first to avoid data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.06, random_state=42)

# Prepare the unsold data for evaluation
X_unsold = df_unsold.drop(['reiwa_price', 'reiwa_is_sold', 'identifier'], axis=1)
y_unsold = df_unsold['reiwa_price']

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train, nthread=-1)
dtest = xgb.DMatrix(X_test, label=y_test, nthread=-1)
dunsold = xgb.DMatrix(X_unsold, label=y_unsold, nthread=-1)

In [19]:
# Custom callback to update tqdm progress bar
class TqdmCallback(xgb.callback.TrainingCallback):
    def __init__(self, total_rounds, early_stopping_rounds):
        self.pbar = tqdm(total=total_rounds, desc="Training Progress")
        self.early_stopping_rounds = early_stopping_rounds
        self.best_score = float("inf")
        self.best_iteration = 0
        self.stopping_counter = 0

    def after_iteration(self, model, epoch, evals_log):
        train_rmse = evals_log['train']['rmse'][-1]
        eval_rmse = evals_log['eval']['rmse'][-1]
        unsold_rmse = evals_log['unsold']['rmse'][-1]

        self.pbar.set_postfix({
            'train_rmse': f'{train_rmse:.5f}', 
            'eval_rmse': f'{eval_rmse:.5f}',
            'unsold_rmse': f'{unsold_rmse:.5f}',
            'best_iteration': self.best_iteration
        })
        self.pbar.update(1)

        # Early stopping logic
        if eval_rmse < self.best_score:
            self.best_score = eval_rmse
            self.best_iteration = epoch
            self.stopping_counter = 0
        else:
            self.stopping_counter += 1

        if self.stopping_counter >= self.early_stopping_rounds:
            self.pbar.set_postfix_str(f'Early stopping at iteration {self.best_iteration} with best score {self.best_score}')
            self.pbar.close()
            return True  # Return True to stop training
        return False
    

# Train the model with early stopping and progress bar
total_rounds = 10000
early_stopping_rounds = 50

# Hyperparameter grid
param_grid = {
    'learning_rate': [0.05],
    'max_depth': [9],
    'min_child_weight': [1],
    'gamma': [0],
    'subsample': [1],
    'colsample_bytree': [1],
    'lambda': [0.1],
    'alpha': [0.1]
}

evals = [(dtrain, 'train'), (dtest, 'eval'), (dunsold, 'unsold')]

param_combinations = list(itertools.product(
    param_grid['learning_rate'],
    param_grid['max_depth'],
    param_grid['min_child_weight'],
    param_grid['gamma'],
    param_grid['subsample'],
    param_grid['colsample_bytree'],
    param_grid['lambda'],
    param_grid['alpha']
))

# Initialize variables to store the best model and lowest unsold RMSE
best_model = None
lowest_unsold_rmse = float('inf')

# Initialize dictionaries to store RMSE values for each hyperparameter
rmse_values = {param: {} for param in param_grid.keys()}

# Loop through each combination of hyperparameters
for params in param_combinations:
    xgb_params = {
        'tree_method': 'hist',
        'eval_metric': 'rmse',
        'device': 'cuda',
        'learning_rate': params[0],
        'max_depth': params[1],
        'min_child_weight': params[2],
        'gamma': params[3],
        'subsample': params[4],
        'colsample_bytree': params[5],
        'lambda': params[6],
        'alpha': params[7]
    }

    # Custom callback to update tqdm progress bar
    class TqdmCallback(xgb.callback.TrainingCallback):
        def __init__(self, total_rounds, early_stopping_rounds):
            self.pbar = tqdm(total=total_rounds, desc="Training Progress")
            self.early_stopping_rounds = early_stopping_rounds
            self.best_score = float("inf")
            self.best_iteration = 0
            self.stopping_counter = 0

        def after_iteration(self, model, epoch, evals_log):
            train_rmse = evals_log['train']['rmse'][-1]
            eval_rmse = evals_log['eval']['rmse'][-1]
            unsold_rmse = evals_log['unsold']['rmse'][-1]

            self.pbar.set_postfix({
                'train_rmse': f'{train_rmse:.5f}', 
                'eval_rmse': f'{eval_rmse:.5f}',
                'unsold_rmse': f'{unsold_rmse:.5f}',
                'best_iteration': self.best_iteration
            })
            self.pbar.update(1)

            # Early stopping logic
            if eval_rmse < self.best_score:
                self.best_score = eval_rmse
                self.best_iteration = epoch
                self.stopping_counter = 0
            else:
                self.stopping_counter += 1

            if self.stopping_counter >= self.early_stopping_rounds:
                self.pbar.set_postfix_str(f'Early stopping at iteration {self.best_iteration} with best score {self.best_score}')
                self.pbar.close()
                return True  # Return True to stop training
            return False

    tqdm_callback = TqdmCallback(total_rounds, early_stopping_rounds)

    min_price = y.min()
    max_price = y.max()

    wrmse_low = np.percentile(y_train, 10)
    wrmse_high = np.percentile(y_train, 90)

    def weighted_rmse(preds, dtrain):
        y_true = dtrain.get_label()
        residuals = preds - y_true
        weights = np.where((y_true <= wrmse_low) | (y_true >= wrmse_high), 2.5, 1) # Prioritize head/tail prediction accuracy
        weights += np.where((y_true <= min_price) | (y_true >= max_price), 100, 0) # Prioritize predicting inside 50k - 10m
        weighted_residuals = weights * residuals
        grad = 2 * weighted_residuals
        hess = 2 * weights
        return grad, hess

    xgb_model = xgb.train(
        params=xgb_params, 
        dtrain=dtrain, 
        num_boost_round=total_rounds, 
        evals=evals, 
        obj=weighted_rmse, 
        callbacks=[tqdm_callback, xgb.callback.EarlyStopping(rounds=early_stopping_rounds)], 
        verbose_eval=0
    )


    # Close the progress bar properly after training
    tqdm_callback.pbar.close()

    # Calculate RMSE on unsold data
    unsold_predictions = xgb_model.predict(dunsold)
    unsold_rmse = np.sqrt(mean_squared_error(y_unsold, unsold_predictions))

    # Store the RMSE for each individual hyperparameter
    for param_name, param_value in zip(param_grid.keys(), params):
        if param_value not in rmse_values[param_name]:
            rmse_values[param_name][param_value] = []
        rmse_values[param_name][param_value].append(unsold_rmse)

    # Check if this model has the lowest unsold RMSE
    if unsold_rmse < lowest_unsold_rmse:
        lowest_unsold_rmse = unsold_rmse
        best_model = xgb_model
        best_params = xgb_params

    print(f"Params: {xgb_params}, Unsold RMSE: {unsold_rmse:.8f}")

# Output the best model and its parameters
print(f"Best Parameters: {best_params}")
print(f"Lowest Unsold RMSE: {lowest_unsold_rmse:.8f}")

# Calculate and print average RMSE for each hyperparameter in condensed format
for param_name, param_values in rmse_values.items():
    for param_value, rmses in sorted(param_values.items()):
        avg_rmse = np.mean(rmses)
        print(f"{param_name} - {param_value} - {avg_rmse:.8f}")

# Save the best XGBoost model
best_model.save_model('xgb_model.json')

# Print results for the best model on test/validation data
xgb_test_predictions = best_model.predict(dtest)
xgb_valid_predictions = best_model.predict(dunsold)
xgb_mae = mean_absolute_error(y_test, xgb_test_predictions)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_test_predictions))

print(f"XGBoost - MAE: {xgb_mae:.8}, RMSE: {xgb_rmse:.8f}")

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
# Add adjusted predictions to the unsold data
df_unsold.loc[:, 'model_prediction'] = inv_boxcox(xgb_valid_predictions, boxcox_lambda)

# Merge the predictions with property_data
df_target = pd.merge(property_df, df_unsold[['identifier', 'model_prediction']], on='identifier', how='left')

# Reorder columns to place model_prediction as the third column
cols = list(df_target.columns)
prediction_index = cols.index('model_prediction')
if prediction_index != 2:
    cols.insert(2, cols.pop(prediction_index))
df_target = df_target[cols]

# Remove rows with no predicted price
df_target = df_target.dropna(subset=['model_prediction'])

# Convert the cleaned DataFrame to JSON format
df_target_json = df_target.to_dict(orient='records')

# Save the JSON to a file
with open('property_data_unsold_predictions.json', 'w') as json_file:
    json.dump(df_target_json, json_file, indent=4)

# Display a few entries from the JSON file
print(df_target_json[:5])

# Notify desktop that the notebook is complete
notify_when_done('Jupyter', 'Notebook execution has completed the final cell.')


[{'reiwa_address': '27 Hovea Court, Morley', 'reiwa_price': 689000.0, 'model_prediction': 798097.5625, 'reiwa_landsize': 710.0, 'reiwa_latitude': -31.8722835, 'reiwa_longitude': 115.9294993, 'reiwa_bedrooms': 4, 'reiwa_bathrooms': 2, 'reiwa_parking': 1, 'reiwa_house_type': 'House', 'reiwa_image_url': 'https://imagecdn.reiwa.com.au/listing/25/4810225-01.jpg?maxwidth=800&maxheight=600&quality=80', 'reiwa_details_url': '/27-hovea-court-morley-4810225/', 'reiwa_is_sold': False, 'reiwa_floor_plan_count': 1, 'reiwa_agency_name': 'Century 21 Gold Key Realty', 'reiwa_agency_no': 13603, 'reiwa_pets_allowed': False, 'reiwa_suburb': 'Morley', 'reiwa_listing_id': 11826361, 'osm_local_community_population': 13075, 'osm_local_community_dwellings': 5187, 'osm_distance_to_perth_cbd': 10.87875397060591, 'osm_distance_to_perth_airport': 8.177887230719264, 'osm_nearest_fuel_station': 0.5511484966103439, 'osm_nearest_bus_stop': 0.17411046698582097, 'osm_nearest_train_station': 3.439569899500056, 'osm_near

<IPython.core.display.Javascript object>